In [1]:
import matplotlib as mpl #画图用的库
import matplotlib.pyplot as plt
#下面这一句是为了可以在notebook中画图
%matplotlib inline
import numpy as np
import sklearn   #机器学习算法库
import pandas as pd #处理数据的库   
import os
import sys
import time
import tensorflow as tf
 
from tensorflow import keras   #使用tensorflow中的keras
#import keras #单纯的使用keras
 
print(tf.__version__)
print(sys.version_info)
for module in mpl, np, sklearn, pd, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.18.0
sklearn 0.21.3
pandas 0.25.3
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [17]:
#input_signature表示 输入签名，即对输入的参数类型做出限定
@tf.function(input_signature=[tf.TensorSpec([None,],tf.int32,name='x')])
def cube(z):
    return tf.pow(z, 3)

In [18]:
#@tf.function py func -> graph
#get_concreate_function -> add input signature -> SavedModel
#使用get_concreate_function可以将上面的@tf.function加上input signature，从而可以使用 SavedModel

cube_func_int32 = cube.get_concrete_function(tf.TensorSpec([None],tf.int32))
print(cube_func_int32)

print(cube_func_int32 is cube.get_concrete_function(tf.TensorSpec([5],tf.int32)))
print(cube_func_int32 is cube.get_concrete_function(tf.constant([1,2,3])))

cube_func_int32.graph

'\ncube_func_int32 = cube.get_concrete_function(tf.TensorSpec([None],tf.int32))\nprint(cube_func_int32)\n\nprint(cube_func_int32 is cube.get_concrete_function(tf.TensorSpec([5],tf.int32)))\nprint(cube_func_int32 is cube.get_concrete_function(tf.constant([1,2,3])))\n\ncube_func_int32.graph\n'

In [19]:
print(cube(tf.constant([1,2,3])))

tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [20]:
#将签名函数保存为SavedModel
to_export = tf.Module()
to_export.cube = cube

tf.saved_model.save(to_export,'./signature_to_saved_model')

INFO:tensorflow:Assets written to: ./signature_to_saved_model/assets


In [21]:
!saved_model_cli show --dir ./signature_to_saved_model --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['x'] tensor_info:
        dtype: DT_INT32
        shape: (-1)
        name: serving_default_x:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_INT32
        shape: (-1)
        name: PartitionedCall:0
  Method name is: tensorflow/serving/predict


In [22]:
!saved_model_cli show --dir ./signature_to_saved_model --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['x'] tensor_info:
      dtype: DT_INT32
      shape: (-1)
      name: serving_default_x:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['output_0'] tensor_info:
      dtype: DT_INT32
      shape: (-1)
      name: PartitionedCall:0
Method name is: tensorflow/serving/predict


In [24]:
imported = tf.saved_model.load('./signature_to_saved_model')
imported.cube(tf.constant([2]))

<tf.Tensor: id=385, shape=(1,), dtype=int32, numpy=array([8], dtype=int32)>